# Projeto back-app

## Preparando o acesso ao Ambiente

In [ ]:
export KUBECONFIG=~/work/kubeconfig
export myNamespace=kdop-learn
cd ~/work/inovacao
echo $PWD

1. Publicando o Projeto back-app

In [ ]:
helm upgrade --install --namespace $myNamespace --create-namespace back-app ~/work/inovacao/back-app/helm/back-app

2. Consultando os recursos do namespace

In [ ]:
kubectl get all -n $myNamespace

In [27]:
# Estendendo as informações sobre os recursos
kubectl get pods -o wide

NAME                         READY   STATUS    RESTARTS   AGE   IP             NODE                                NOMINATED NODE   READINESS GATES
back-app-6d46745ffc-g2zkc    1/1     Running   0          84m   10.240.0.10    aks-agentpool-65799894-vmss00000h   <none>           <none>
cert-app-657b767888-bt5pr    1/1     Running   0          60m   10.240.0.13    aks-agentpool-65799894-vmss00000h   <none>           <none>
cert-app-657b767888-knbm8    1/1     Running   0          16m   10.240.0.105   aks-agentpool-65799894-vmss00000h   <none>           <none>
front-app-76789b56bf-9gvwf   1/1     Running   0          82m   10.240.0.87    aks-agentpool-65799894-vmss00000h   <none>           <none>


In [28]:
kubectl get pods --show-labels

NAME                         READY   STATUS    RESTARTS   AGE    LABELS
back-app-6d46745ffc-g2zkc    1/1     Running   0          137m   app.kubernetes.io/instance=back-app,app.kubernetes.io/name=back-app,pod-template-hash=6d46745ffc
cert-app-657b767888-bt5pr    1/1     Running   0          113m   app.kubernetes.io/instance=cert-app,app.kubernetes.io/name=cert-app,pod-template-hash=657b767888
cert-app-657b767888-knbm8    1/1     Running   0          68m    app.kubernetes.io/instance=cert-app,app.kubernetes.io/name=cert-app,pod-template-hash=657b767888
front-app-76789b56bf-9gvwf   1/1     Running   0          135m   app.kubernetes.io/instance=front-app,app.kubernetes.io/name=front-app,pod-template-hash=76789b56bf


In [48]:
kubectl get pod -l app.kubernetes.io/name=front-app

NAME                         READY   STATUS    RESTARTS   AGE
front-app-76789b56bf-9gvwf   1/1     Running   0          154m


In [47]:
kubectl describe pod -l app.kubernetes.io/name=front-app

Name:         front-app-76789b56bf-9gvwf
Namespace:    kdop-learn
Priority:     0
Node:         aks-agentpool-65799894-vmss00000h/10.240.0.4
Start Time:   Mon, 09 Aug 2021 19:29:26 +0000
Labels:       app.kubernetes.io/instance=front-app
              app.kubernetes.io/name=front-app
              pod-template-hash=76789b56bf
Annotations:  <none>
Status:       Running
IP:           10.240.0.87
IPs:
  IP:           10.240.0.87
Controlled By:  ReplicaSet/front-app-76789b56bf
Containers:
  front-app:
    Container ID:   containerd://350ec836d46e47b86b3d29088d7e3546fe0544eb2254feaac26ccd29fcdb496e
    Image:          kdop/front-app:0.0.5
    Image ID:       docker.io/kdop/front-app@sha256:7eadf38a58d62aef00e97234fda9ecfbaced421c569eda9b088cf50d05c2930c
    Port:           5000/TCP
    Host Port:      0/TCP
    State:          Running
      Started:      Mon, 09 Aug 2021 19:29:28 +0000
    Ready:          True
    Restart Count:  0
    Limits:
      cpu:     50m
      memory:  128Mi
    Req

In [38]:
kubectl logs -l 'app.kubernetes.io/name=front-app' --all-containers

[pid: 10|app: 0|req: 282/1768] 10.240.0.4 () {30 vars in 351 bytes} [Mon Aug  9 21:55:57 2021] GET /health => generated 31 bytes in 4 msecs (HTTP/1.1 200) 2 headers in 79 bytes (1 switches on core 0)
[pid: 11|app: 0|req: 769/1769] 10.240.0.4 () {30 vars in 351 bytes} [Mon Aug  9 21:55:59 2021] GET /health => generated 31 bytes in 3 msecs (HTTP/1.1 200) 2 headers in 79 bytes (1 switches on core 0)
[pid: 9|app: 0|req: 264/1770] 10.240.0.4 () {30 vars in 351 bytes} [Mon Aug  9 21:56:07 2021] GET /health => generated 31 bytes in 4 msecs (HTTP/1.1 200) 2 headers in 79 bytes (1 switches on core 0)
[pid: 11|app: 0|req: 770/1771] 10.240.0.4 () {30 vars in 351 bytes} [Mon Aug  9 21:56:09 2021] GET /health => generated 31 bytes in 4 msecs (HTTP/1.1 200) 2 headers in 79 bytes (1 switches on core 0)
[pid: 8|app: 0|req: 239/1772] 10.240.0.4 () {30 vars in 351 bytes} [Mon Aug  9 21:56:17 2021] GET /health => generated 31 bytes in 4 msecs (HTTP/1.1 200) 2 headers in 79 bytes (1 switches on core 0)
[p

3. Validando o Serviço

    Para acessar o back-app fora do Cluster, necessitamos expor temporariamente o serviço
    
    Abra um terminal e execute o comando abaixo
```bash
export KUBECONFIG=~/kubeconfig
kubectl port-forward svc/back-app 8080:5000 -n default 
```

In [ ]:
curl http://localhost:8080/health

In [44]:
# Alternativas para executar comandos em um pod
kubectl exec svc/front-app -- curl -s http://localhost:5000/health

Front-app in ns kdop-learn: Ok!

In [45]:
kubectl exec deploy/front-app -- curl -s http://localhost:5000/health

Front-app in ns kdop-learn: Ok!

Agora que os PODs estão em execução, podemos também validar a aplicação front-app.  


In [26]:
echo http://learn.kdop.net/$myNamespace/front-app/health
echo http://learn.kdop.net/$myNamespace/front-app/index

http://learn.kdop.net/kdop-learn/front-app/health
http://learn.kdop.net/kdop-learn/front-app/index


### ConfigMap

Como o front-app acessa o back-app?

In [ ]:
kubectl describe cm/front-app -n $myNamespace

### HPA

```yaml
apiVersion: autoscaling/v2beta1
kind: HorizontalPodAutoscaler
metadata:
  name: back-app
  labels:
    back-app
spec:
  scaleTargetRef:
    apiVersion: apps/v1
    kind: Deployment
    name: back-app
  minReplicas: 1
  maxReplicas: 3
  metrics:
    - type: Resource
      resource:
        name: cpu
        targetAverageUtilization: 80
    - type: Resource
      resource:
        name: memory
        targetAverageUtilization: 80
```

In [ ]:
kubectl get hpa -n $myNamespace

### Volumes

O conceito de volume para PODs é o mesmo que para máquinas VM ou físicas, é uma área de armazenamento que será montada em um diretório ou em uma letra (windows).

![](media/aks-storage.png)

Fonte: [Opções de armazenamento para aplicativos no Serviço de Kubernetes do Azure (AKS)](https://docs.microsoft.com/pt-br/azure/aks/concepts-storage)

Existem muitos tipos de armazenamentos, os mais comuns são os efêmeros e os persistentes.

* [Efêmeros](https://kubernetes.io/docs/concepts/storage/ephemeral-volumes/): são os mais comuns e são perdidos quando o POD e reiniciado
* [Persistente](https://kubernetes.io/docs/concepts/storage/persistent-volumes/): Como o nome diz, são mantidos mesmo durante reinicio dos PODs. O Kubernetes encontra automaticamente os volumes correspondentes aos PODs.

Consulte [Kubernetes - Volumes](https://kubernetes.io/docs/concepts/storage/), para mais  opções. Além dos tipos, cada sistema oferece tipos diferentes de armazenamentos, tais como SSDs, discos mecânicos, etc. No kubernetes isso é representado pela classe de aramazenamento (Storage Class). Uma classe de armazenamento (SC) é utilizada para definir o tipo de volume persistente (PV) que será criado automaticamente a partir de um requerimento (PVC).

Para conhecer as classes de armazenamento disnponíveis no seu cluster, utilize o comando:

In [ ]:
kubectl get sc

Os nomes mudam de acordo com o lugar que o seu cluster está instalado, a lista acima são as classes da Azure.

Você pode omitir a classe de armazenamento, neste caso será utilizada a classe padrão.

A nossa aplicação é uma espécie de banco de dados rudimentar que precisa armazenar seus dados de forma persistente, para tal configuramos um requerimento de volume persistente.

Vejamos a configuração:

In [ ]:
helm template --namespace $myNamespace --create-namespace back-app ~/work/inovacao/back-app/helm/back-app

Selecionamos o trecho referente ao volume:

```yaml
---
# Source: back-app/templates/pcv.yaml
kind: PersistentVolumeClaim
metadata:
  name: back-app-managed-disk
spec:
  accessModes:
  - ReadWriteOnce
  storageClassName: managed-premium
  resources:
    requests:
      storage: 1Gi
---
# Source: back-app/templates/deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: back-app
  labels:
    helm.sh/chart: back-app-0.1.0
    app.kubernetes.io/name: back-app
    app.kubernetes.io/instance: back-app
    app.kubernetes.io/version: "1.16.0"
    app.kubernetes.io/managed-by: Helm
spec:
  replicas: 1
  selector:
    matchLabels:
      app.kubernetes.io/name: back-app
      app.kubernetes.io/instance: back-app
  template:
    metadata:
      labels:
        app.kubernetes.io/name: back-app
        app.kubernetes.io/instance: back-app
    spec:
      serviceAccountName: back-app
      securityContext:
        {}
      containers:
        - name: back-app
          securityContext:
            {}
          image: "kdop/back-app:0.0.1"
          imagePullPolicy: Always
          ports:
            - name: http
              containerPort: 5000
              protocol: TCP
          livenessProbe:
            httpGet:
              path: /health
              port: http
          readinessProbe:
            httpGet:
              path: /health
              port: http
          resources:
            limits:
              cpu: 200m
              memory: 256Mi
            requests:
              cpu: 200m
              memory: 256Mi
          volumeMounts:
          - mountPath: "/tmp/inovacao/back-app"
            name: back-volume
      volumes:
        - name: back-volume
          persistentVolumeClaim:
            claimName: back-app-managed-disk
```

Um `PersistentVolume` foi criado dinamicamente em resposta ao nosso `PersistentVolumeClaim`. O K8s interage com as APIs da Azure para criar o PV na forma de um disco.

Você pode criar seus `PersistentVolume` e/ou associá-los a discos já existentes.

In [49]:
kubectl get pv --sort-by=.spec.capacity.storage

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                                 STORAGECLASS      REASON   AGE
pvc-d813c6c6-7e3e-447b-a9f0-ef9dd4a07702   1Gi        RWO            Delete           Bound    kdop-learn/back-app-managed-disk      managed-premium            3h
pvc-1c507ed7-f0d5-4622-8c67-6b6a2a155c85   32Gi       RWO            Delete           Bound    kubecost/kubecost-prometheus-server   default                    90d
pvc-fee03536-ab2c-4de4-a475-2c01f1069899   32Gi       RWO            Delete           Bound    kubecost/kubecost-cost-analyzer       default                    90d


In [51]:
# Campos do PV
kubectl get pv pvc-d813c6c6-7e3e-447b-a9f0-ef9dd4a07702 -o yaml

apiVersion: v1
kind: PersistentVolume
metadata:
  annotations:
    pv.kubernetes.io/bound-by-controller: "yes"
    pv.kubernetes.io/provisioned-by: kubernetes.io/azure-disk
    volumehelper.VolumeDynamicallyCreatedByKey: azure-disk-dynamic-provisioner
  creationTimestamp: "2021-08-09T19:27:28Z"
  finalizers:
  - kubernetes.io/pv-protection
  labels:
    failure-domain.beta.kubernetes.io/region: eastus
    failure-domain.beta.kubernetes.io/zone: eastus-1
  name: pvc-d813c6c6-7e3e-447b-a9f0-ef9dd4a07702
  resourceVersion: "2121685"
  uid: 9f50d7c0-6976-4f0e-b873-7dd9f28af381
spec:
  accessModes:
  - ReadWriteOnce
  azureDisk:
    cachingMode: ReadOnly
    diskName: kubernetes-dynamic-pvc-d813c6c6-7e3e-447b-a9f0-ef9dd4a07702
    diskURI: /subscriptions/73c8b399-a4ed-44b8-af3f-081a94ca1f65/resourceGroups/mc_kdop-dev_kdop-learn_eastus/providers/Microsoft.Compute/disks/kubernetes-dynamic-pvc-d813c6c6-7e3e-447b-a9f0-ef9dd4a07702
    fsType: ""
    kind: Managed
    readOnly: false
  capacity:

O comando `get pv` listou um `PersistentVolumes` associado ao seu requerimento.

Na figura abaixo, você pode ver os dois discos na Azure.

![](media/azure-disks.png)

Se você descrever o `PersistentVolume` associado verá alguns labels que descrevem sua origem.

In [ ]:
kubectl describe pv/pvc-d813c6c6-7e3e-447b-a9f0-ef9dd4a07702

Um olhar mais detalhado no disco na Azure revelará como o Kubernetes faz sua mágica para encontrar seus volumes e seus respectivos donos.

![](media/azure-disk.png)

Alguns desses labels também aparecem no `PersistentVolumeClaim`.

In [ ]:
kubectl describe pvc/back-app-managed-disk -n $myNamespace

[<img align="left" src="media/voltar.png" width="100"/>](03_Helm.ipynb) [<img align="right" src="media/avancar.png" width="100"/>](05_Cert.ipynb)